In [1]:
import pandas as pd
import joblib
import argparse

In [ ]:
# Create the parser
parser = argparse.ArgumentParser(prog="BayesienReader",description="Creer un modele bayesien avec une base de donnée sous format csv séparer par tabulation en premier l'étiquette 'ham' ou 'spam' et le contenue du sms")

# Add arguments
parser.add_argument('--TFile', type=str, help="The path to the training data", required=True)
parser.add_argument('--OFile',type = str,help="The name of the outputfile")
# Parse arguments
args = parser.parse_args()

In [6]:
loaded_df, spamProbality = joblib.load('BayesienModel.joblib')

In [7]:
def TestPhraseList(phraseList,spamProba=1):
    for word in phraseList:
        vraisemblance = loaded_df["vraisemblance"].get(word,"NotFound")
        if vraisemblance!="NotFound":
            spamProba*=vraisemblance
    return(spamProba)

def TestPhrase(phrase:str,spamProba=1):
    phraseList=phrase.split()
    return TestPhraseList(phraseList,spamProba=spamProba)


In [8]:
testDF = pd.read_csv(
    "SMSSpamCollection.txt", delimiter="\t", header=None, names=["spam", "text"]
)

In [9]:
testDF["text"]=testDF["text"].str.split()
testDF["result"]=testDF["text"].apply(TestPhraseList,spamProba=spamProbality)
testDF["Finalresult"]=['spam' if x > 1 else 'ham' for x in testDF['result']]

In [10]:
wrong = len(testDF[testDF["Finalresult"]!=testDF["spam"]].index)
FP = len(testDF[(testDF["Finalresult"]=="spam")&(testDF["spam"]=="ham")].index)
FN = len(testDF[(testDF["Finalresult"]=="ham")&(testDF["spam"]=="spam")].index)
VP = len(testDF[(testDF["Finalresult"]=="ham")&(testDF["spam"]=="ham")].index)
VN = len(testDF[(testDF["Finalresult"]=="spam")&(testDF["spam"]=="spam")].index)
wrong/len(testDF.index)
exactitude = (VN+VP)/(VN+FN+FP+VP)
print(f"Exactitude : {exactitude}")
rappel = VP /(VP+FN)
print(f"Rappel : {rappel}")
precision = VP/(VP+FP)
print(f"Precision : {precision}")
F1Score = (2 * precision * rappel) / (precision + rappel)
print(f"F1-Score : {F1Score}")

Exactitude : 0.9800789662598708
Rappel : 0.9972573839662447
Precision : 0.9796891191709844
F1-Score : 0.988395190799791
